## Music Market and Leading to our company marketing strategy 

Everyone loves music and of course music is a part of our lives. Many cultures and people define theirself with music. Songs, albums, music consumers and producers have created a huge music industry. The music industry has also produced very big data about singer, albums, songs, customer, buying and selling, etc.  Let's look close the music world's data over Chinook Database.

In [52]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Chinook database has many tables that contains data. You may see the database schema of the tables below:

![The Chinook Database Schema](schema.png)

I don't want to confuse you with the schema. You just need to know that there are tables and that they are related to each other. You can see table list of chinook database below:

In [53]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Let's look closer to the data. For example, let's take a look at the total sales of Queen's albums in the United States. 

In [54]:
%%sql
SELECT 
    ar.name artist,
    al.title album_title,
    strftime('%Y', i.invoice_date) sales_date,
    SUM(il.quantity) sales_volume,
    "$" || SUM(i.total) sales_revenue
    FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id 
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN album al ON al.album_id = t.album_id
        INNER JOIN artist ar ON ar.artist_id = al.artist_id
        WHERE country = "USA" AND ar.name = "Queen"
    GROUP BY 2,3
    ORDER BY 3 DESC; 

Done.


artist,album_title,sales_date,sales_volume,sales_revenue
Queen,Greatest Hits I,2020,1,$9.9
Queen,Greatest Hits II,2020,2,$17.82
Queen,Greatest Hits I,2019,2,$19.8
Queen,Greatest Hits II,2019,4,$33.66
Queen,Greatest Hits I,2018,1,$10.89
Queen,Greatest Hits II,2018,2,$17.82
Queen,News Of The World,2018,1,$2.97
Queen,Greatest Hits I,2017,4,$32.67
Queen,Greatest Hits II,2017,2,$20.79


### Genre Analysis of Our Sales in USA
We're interested in finding out which genres sell the best in the USA. We'll need to write a query to find out which genres sell the most tracks in the USA, and make a recommendation for the three artists whose albums we should purchase for the store.

In [55]:
%%sql
WITH total_sold AS (
                    SELECT * FROM invoice_line il
                    INNER JOIN invoice i on il.invoice_id = i.invoice_id
                    INNER JOIN customer c on i.customer_id = c.customer_id
                    WHERE c.country = "USA"
                    )
SELECT 
    g.name genre,
    COUNT(ts.quantity) tracks_sold,
    cast(count(ts.quantity) AS FLOAT) / 
                (SELECT COUNT(*) from total_sold) percentage_sold
    FROM total_sold ts
        INNER JOIN track t ON t.track_id = ts.track_id
        INNER JOIN genre g ON g.genre_id = t.genre_id        
    GROUP BY 1
    ORDER BY 2 DESC; 

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


As seen from the sales figures, the pieces in the Rock, Punk and Metal genres are the most purchased music pieces. 

When R&B/Soul (53), Blues (36), Jazz (14) and similar genres are evaluated together, it is seen that the 4th largest segment is an important market size in R&B and Jazz tours. And even when alternative and easy listening are taken into account, it is easily seen that there is an important audience that likes softer music, unlike rock and metal. Together with the smallness of the market share of each, it is seen that as a total of Nich markets, it contains important opportunities.

What should our strategy be here? These data can be evaluated differently depending on our company's company strategies. Considering the seasonal trends and customer characteristics, investment in Rock, Punk and Metal genres is considered profitable. However, if long-term trains, creating long-term new customer base formation and niche opportunities are considered together, investing in RBJ can increase product diversity and we remain competitive. 

#### Employee Performance Evaluation

Let's look at the total dollar amount of sales assigned to each sales support agent within the company. 

In [56]:
%%sql
WITH sales_info AS (
                    SELECT * FROM customer c
                    INNER JOIN invoice i on i.customer_id = c.customer_id
                    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
                    )

SELECT 
    e.first_name || " " || e.last_name name,
    strftime('%Y', e.hire_date) hire_date,
    ROUND(SUM(si.total), 2) sales_dolar,
    SUM(si.quantity) sold_tracks,
    ROUND(cast(SUM(si.total) AS FLOAT) / SUM(si.quantity), 2) performance_per_track
    FROM sales_info si
        INNER JOIN employee e ON si.support_rep_id = e.employee_id
    GROUP BY 1
    ORDER BY 2 DESC;


Done.


name,hire_date,sales_dolar,sold_tracks,performance_per_track
Jane Peacock,2017,18246.69,1749,10.43
Margaret Park,2017,15923.16,1600,9.95
Steve Johnson,2017,13333.32,1408,9.47


Three of the employee performed all sales. According to evaluation between them; Though all of them was hired same year, Jane Peacock has best sale performance at the company. Jane Peacock's performance is %8.5 better then next employee's and %19.45 better then 3rd one's performanse.

### Customers Lifetime Value nased Country and Future Market Analysis

We want to evaluate our customers life time value for each country based on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

There are a number of countries with only one customer. So we will group that countries as "Other" and analyze the countries over their customer numbers, total sales, average sales per customer and average order.

In [85]:
%%sql
DROP VIEW customers_country;

CREATE VIEW customers_country AS
    SELECT
        country origin,
        CASE
             WHEN COUNT(distinct c.customer_id) > 1 THEN c.country
             ELSE "OTHER"
        END AS country,
        COUNT(distinct c.customer_id) counts,
        COUNT(distinct i.invoice_id) invoices,
        SUM(i.total) total_sales
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 2 DESC;


SELECT
    country,
    SUM(counts) customer_num,
    ROUND(total_sales, 2) total_sales,
    ROUND(SUM(total_sales)/SUM(counts), 2) average_sales_p_customer,
    ROUND(SUM(total_sales)/SUM(invoices), 2) average_order

FROM
    customers_country cc
    
GROUP BY 1
ORDER BY total_sales DESC;



Done.
Done.
Done.


country,customer_num,total_sales,average_sales_p_customer,average_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
OTHER,15,75.24,73.0,7.45


According to table, the noteworthy countries are Czech Republic, Germany, United Kingdom and India.
But these data may be deceptive to make decision for those countries. Especially for India with 1.5 billion population and Czech Republic with 10 million, 2 customers are not seen enough for us to make a healthy decision. the sample size is not large enough to give us high confidence. However, it can be said that Czech republic is the most remarkable country in terms of total sales figures, order size per customer and average order size relative to population and customer number.w 

When evaluated together with the previous table, Germany and the United Kingdom can be a safe harbor for familiarity with the musical genre and the values in the current table. It may be helpful to organize more extensive survey studies in these countries. 

### Our Customers' Purchasing Behaviour Pattern
We have analyze our employees performance, our customers purchase and country opportunities. We need to analyze our products and detailed demands of our customers. In order to decrease costs, our company can consider changing our purchasing strategy to save money. The strategy we are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. First of all, we need to learn whether our customers' purchasing pattern is to buy full album or individual tracks from the data.

In [84]:
%%sql
WITH first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     INNER JOIN track t ON t.track_id = il.track_id
     GROUP BY 1
    )


SELECT
    purchased_full_album,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percent
FROM
    (
    SELECT
        ft.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ft.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ft.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ft.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ft.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "purchased_full_album"
     FROM first_track ft
    )
GROUP BY purchased_full_album;

Done.


purchased_full_album,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


According to the result above, full album sales make up only 18.6% of current sales. 81.4% are the orders consisting of individually selected tracks. Accordingly, our customer base consists of conscious individuals who act selectively and pay attention to their budget. In that case, purchasing the pieces liked from the albums with deeper analysis will significantly reduce our costs. With the profit obtained, wider marketing campaigns can be organized, profitable customer base can be increased and our product range can be expanded. 

#### Which artist is used in the most playlists?


In [90]:
%%sql
SELECT
    ar.name artist_name,
    COUNT(ar.name)
    FROM artist ar
    INNER JOIN album a ON a.artist_id = ar.artist_id
    INNER JOIN track t ON t.album_id = a.album_id
    INNER JOIN playlist_track pt ON pt.track_id = t.track_id
    INNER JOIN playlist pl ON pl.playlist_id = pt.playlist_id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10;

Done.


artist_name,COUNT(ar.name)
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Eric Clapton,145
Faith No More,145
Lenny Kravitz,143


The artist most used in the playlists is Iron Maiden.

#### How many tracks have been purchased vs not purchased?


In [128]:
%%sql
WITH trackspurchased AS
    (
    SELECT
        distinct t.track_id
    FROM invoice_line il
    INNER JOIN track t on il.track_id = t.track_id
    )

SELECT
    COUNT(*) purchased_tracks,
    ((SELECT COUNT(*) FROM track) - COUNT(*)) unpurchased_tracks,
    (SELECT COUNT(*) FROM track) total_track,
    ROUND(100*CAST(((SELECT COUNT(*) FROM track) - COUNT(*)) AS FLOAT) / (SELECT COUNT(*) FROM track), 1) Percent_unpurchased
FROM trackspurchased



Done.


purchased_tracks,unpurchased_tracks,total_track,Percent_unpurchased
1806,1697,3503,48.4


#### Is the range of tracks in the store reflective of their sales popularity?


In [147]:
%%sql
   
WITH genre_sales AS
    (
    SELECT
        g.name Genre,
        TOTAL(il.unit_price) Sales
    FROM invoice_line il
    LEFT JOIN track t ON t.track_id = il.track_id
    LEFT JOIN genre g ON g.genre_id = t.genre_id
    GROUP BY 1
    )

SELECT
    g.name Ggnre,
    COUNT(DISTINCT t.track_id) tracks,
    ROUND(100*CAST(COUNT(DISTINCT t.track_id) AS FLOAT) / 
          CAST((SELECT COUNT(DISTINCT track_id) FROM track) AS FLOAT), 2) percent_tracks,
    ROUND(gs.Sales, 2) sales,
    ROUND(100*gs.Sales / (SELECT TOTAL(total) FROM invoice), 2) percent_sales,
    ROUND(100*gs.Sales / (SELECT TOTAL(total) FROM invoice) - 100*CAST(COUNT(DISTINCT t.track_id) AS FLOAT) / 
          CAST((SELECT COUNT(DISTINCT track_id) FROM track) AS FLOAT)) percent_difference
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN genre_sales gs ON gs.Genre = g.name
GROUP BY 1
ORDER BY 6 DESC;

Done.


Ggnre,tracks,percent_tracks,sales,percent_sales,percent_difference
Rock,1297,37.03,2608.65,55.39,18.0
Metal,374,10.68,612.81,13.01,2.0
R&B/Soul,61,1.74,157.41,3.34,2.0
Alternative,40,1.14,115.83,2.46,1.0
Alternative & Punk,332,9.48,487.08,10.34,1.0
Easy Listening,24,0.69,73.26,1.56,1.0
Blues,81,2.31,122.76,2.61,0.0
Electronica/Dance,30,0.86,54.45,1.16,0.0
Hip Hop/Rap,35,1.0,32.67,0.69,-0.0
Pop,48,1.37,62.37,1.32,-0.0


The range of tracks in the store mostly reflects the popularity of sales except Rock, Latin, etc. For example, 1297 Rock pieces represent 37% of the total pieces, but represent 55% of the sales. it provided 18% more return proportionally. Although Latin pieces were 16.53 percent of the total pieces, their return was only 3.51%. Proportionally, it provided 13% less return. 

#### Do protected vs non-protected media types have an effect on popularity?

In [146]:
%%sql
SELECT
    media_type,
    ROUND(TOTAL(unit_price), 2) total_sales,
    ROUND(100*TOTAL(unit_price) / (SELECT TOTAL(unit_price) FROM invoice_line), 1) percent_total_sales
FROM (
    SELECT
        il.invoice_line_id,
        il.invoice_id,
        il.track_id,
        il.unit_price,
        mt.name,
        CASE
            WHEN mt.name = (SELECT name FROM media_type WHERE name LIKE "%Protected%") THEN "Protected"
            ELSE "Non-protected"
            END
        AS media_type
    FROM invoice_line il
    LEFT JOIN track t ON t.track_id = il.track_id
    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
    )
GROUP BY 1

Done.


media_type,total_sales,percent_total_sales
Non-protected,4274.82,90.8
Protected,434.61,9.2


Tracks under non-protected media types account for 90% of the total sales. Protected tracks are not preferred as much as unprotected ones. 

### Conclusion

Rock, Alternative & Punk, Metal, and R&B/Soul are among the top-selling music genres in the USA. When R&B/Soul, Blues, Jazz  and similar genres are evaluated together, it is seen that the 4th largest segment is an important market size in R&B and Jazz tours. And even when alternative and easy listening are taken into account, it is easily seen that there is an important audience that likes softer music, unlike rock and metal. The addition of these type albums and tracks from American artists could improve the sales performance of the store.

I determined the sales performance of the shop's employees. They were hired same year. The sales performance of employees seems to depend on their hiring dates. Though all of them was hired same year, Jane Peacock has best sale performance at the company.

The noteworthy countries are Czech Republic, Germany, United Kingdom and India. But these data may be deceptive to make decision for those countries. Especially for India with 1.5 billion population and Czech Republic with 10 million, 2 customers are not seen enough for us to make a healthy decision. The sample size is not large enough to give us high confidence. However, it can be said that Czech republic is the most remarkable country in terms of total sales figures, order size per customer and average order size relative to population and customer number. Germany and the United Kingdom can be a safe harbor for familiarity with the musical genre and the values in the current table. We can say that Czech Republic, United Kingdom, and Germany show opportunities for growth.

About 20% of total sales come from whole album purchases. 81.4% are the orders consisting of individually selected tracks. Our company especially should continue to selling individual tracks. Our customer base prefer individual tracks liked. In that case, purchasing the pieces liked from the albums with deeper analysis will significantly reduce our costs. 

The artist Iron Maiden is mostly used in the playlists.
About half of the available tracks in the store remain unpurchased.
The range of tracks in the store mostly reflects the sales performance except Rock, Latin etc. It may rethink about the number of tracks classified under Latin, TV Shows, Drama genres following the observed negative trends in sales. On the other hand, R&B/Soul, Alternative and Blues may offer opportunities. 
Tracks categorized under non-protected media types are sold better than protected types.
    

